# Creación y evaluación de soluciones <a class="anchor" id="top"></a>

Con este cuaderno, entrenará varios modelos por medio de Amazon Personalize, concretamente: 

1. Personalización del usuario: qué elementos son más relevantes para un usuario concreto.
1. Elementos similares: dado un elemento, qué elementos son similares a él.
1. Clasificación personalizada: dado un usuario y una colección de elementos, en qué orden son más relevantes.

## Esquema

1. [Introducción](#intro)
1. [Crear soluciones](#solutions)
1. [Evaluar soluciones](#eval)
1. [Uso de métricas de evaluación](#use)
1. [Almacenamiento de variables útiles](#vars)

## Introducción <a class="anchor" id="intro"></a>

En resumen, los algoritmos en Amazon Personalize (llamados recetas) buscan resolver diferentes tareas que se explican a continuación:

1. **User Personalization (Personalización del usuario)**: nueva versión que admite TODOS los flujos de trabajo de HRNN/necesidades de personalización del usuario, será lo que utilicemos aquí.
1. **HRNN & HRNN-Metadata**: recomienda elementos basados en las interacciones previas del usuario con los elementos.
1. **HRNN-Coldstart**: recomienda nuevos elementos para los que aún no se dispone de datos de interacción.
1. **Personalized-Ranking**: toma una recopilación de elementos y, luego, los ubica en un posible orden de interés mediante un enfoque similar al de una HRNN.
1. **SIMS (elementos similares)**: en base a un elemento, recomienda otros elementos con los que también interactúan los usuarios.
1. **Popularity-Count**: recomienda los elementos más populares, si la HRNN o los metadatos de la HRNN no ofrecen una respuesta. Esta es la devolución predeterminada.

Sin importar cuál sea el caso de uso, todos los algoritmos comparten una base de aprendizaje sobre los datos de interacción entre el usuario y los elementos, la cual se define según 3 atributos principales:

1. **UserID**: el usuario que interactuó
1. **ItemID**: el elemento con el que interactuó el usuario
1. **Timestamp (Marca temporal)**: la hora a la que ocurrió la interacción

También admitimos los tipos de eventos y valores de eventos definidos por lo siguiente:

1. **Tipo de evento**: etiquetado de eventos por categoría (navegación, compra, clasificación, etc.).
1. **Valor de evento**: valor que corresponde al tipo de evento acontecido. En términos generales, buscamos valores normalizados entre 0 y 1 entre los tipos de eventos. Por ejemplo, si existen tres fases para completar una transacción (seleccionar, agregar al carrito y comprar), entonces hay un valor de evento para cada fase, como 0.33, 0.66 y 1.0, respectivamente.

Los campos para el tipo de evento y valor de evento son datos adicionales que se pueden utilizar para filtrar los datos enviados para el entrenamiento del modelo de personalización. En este ejercicio en particular no tendremos un tipo de evento ni un tipo de valor. 

Para ejecutar este cuaderno, es necesario haber ejecutado los cuadernos anteriores, `01_Validating_and_Importing_User_Item_Interaction_Data` y `02_Validating_and_Importing_Item_Metadata.ipynb`, en los que creó un conjunto de datos e importó datos de interacción en Amazon Personalize. Al final de ese cuaderno, guardó algunos de los valores de las variables, que ahora debe cargar en este cuaderno.

In [ ]:
%store -r

## Crear soluciones <a class="anchor" id="solutions"></a>
[Regresar al principio](#top)

En este cuaderno, creará soluciones con las siguientes recetas:

1. User Personalization
1. SIMS
1. Personalized-Ranking

La receta Popularity-Count es la solución más sencilla disponible en Amazon Personalize y solo debería utilizarse como alternativa, por lo que tampoco se tratará en este cuaderno.

Al igual que en el cuaderno anterior, comience por importar los paquetes pertinentes y configure una conexión con Amazon Personalize mediante el SDK.

In [ ]:
import time
from time import sleep
import json

import boto3

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Una variación específica de un algoritmo se denomina receta en Amazon Personalize. Las diferentes recetas son adecuadas para diferentes situaciones. Un modelo entrenado se denomina solución, y cada solución puede tener muchas versiones relacionadas con un determinado volumen de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que son compatibles. Esto le permitirá seleccionar una y utilizarla para crear el modelo.

In [ ]:
personalize.list_recipes()

El resultado es solo una representación JSON de todos los algoritmos mencionados en la introducción.

A continuación, seleccionaremos recetas específicas y crearemos modelos con ellas.

### User Personalization
La receta User-Personalization (aws-user-personalization) está optimizada para todos los escenarios de recomendación USER_PERSONALIZATION. Utiliza la exploración automática de elementos para hacer recomendaciones.

Con la exploración automática, Amazon Personalize prueba automáticamente diferentes recomendaciones de elementos, aprende de la forma en que los usuarios interactúan con estos elementos recomendados y potencia las recomendaciones de elementos que impulsan un mejor compromiso y conversión. Esto mejora el descubrimiento de elementos y la participación cuando tiene un catálogo que cambia rápidamente, o cuando los nuevos elementos, como las noticias o las promociones, son más relevantes para los usuarios cuando están frescos.

Puede equilibrar cuánto explorar (donde los elementos con menos datos de interacciones o relevancia se recomiendan con más frecuencia) frente a cuánto explotar (donde las recomendaciones se basan en lo que conocemos o en la relevancia). Amazon Personalize ajusta automáticamente las futuras recomendaciones en función de los comentarios implícitos de los usuarios.

En primer lugar, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [ ]:
user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Crear la solución

Primero cree una solución con la receta. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

In [ ]:
user_personalization_create_solution_response = personalize.create_solution(
    name = "personalize-poc-userpersonalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = user_personalization_recipe_arn
)

user_personalization_solution_arn = user_personalization_create_solution_response['solutionArn']


In [ ]:
print(json.dumps(user_personalization_solution_arn, indent=2))

#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 90 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [ ]:
userpersonalization_create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

In [ ]:
userpersonalization_solution_version_arn = userpersonalization_create_solution_version_response['solutionVersionArn']
print(json.dumps(user_personalization_create_solution_response, indent=2))

### SIMS


SIMS es uno de los algoritmos más antiguos utilizados en Amazon para los sistemas de recomendación. Un caso de uso básico es cuando se tiene un elemento y se quiere recomendar elementos con los que se ha interactuado de forma similar en toda la base de usuarios. Esto significa que el resultado no se personaliza por usuario. A veces, esto hace que se recomienden sobre todo elementos populares, por eso hay un hiperparámetro que puede ajustarse para reducir los elementos populares en los resultados. 

Para nuestro caso de uso, a partir de los datos de MovieLens, supongamos que elegimos una película concreta. A continuación, podemos utilizar SIMS para recomendar otras películas basándonos en el comportamiento de interacción de toda la base de usuarios. Los resultados no son personalizados por usuario, sino que difieren en función de la película que hayamos elegido como entrada.

Al igual que la última vez, empezamos por seleccionar la receta.

In [ ]:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Crear la solución

Al igual que con la HRNN, hay que empezar por crear la solución primero. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

In [ ]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 35 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [ ]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [ ]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

### Clasificación personalizada

La clasificación personalizada es una aplicación interesante de la HRNN. En lugar de limitarse a recomendar lo más probable para el usuario en cuestión, este algoritmo toma un usuario y también una lista de elementos. A continuación, los elementos se vuelven a presentar en el orden de relevancia más probable para el usuario. El caso de uso aquí es para filtrar en categorías únicas que no tiene metadatos de elementos para crear un filtro, o cuando tiene una amplia colección que le gustaría ordenar mejor para un usuario en particular.

Para nuestro caso de uso, por medio de los datos de MovieLens, podríamos imaginar que una aplicación de video bajo demanda (VOD) podría querer crear un estante de películas de cómics, o películas de un director específico. Lo más probable es que tengamos estas listas basadas en los metadatos de los títulos que tenemos. Utilizaríamos la clasificación personalizada para reordenar la lista de películas de cada usuario, basándonos en su historial de etiquetado anterior. 

Al igual que la última vez, empezamos por seleccionar la receta.

In [ ]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Crear la solución

Al igual que con la solución anterior, comience por crear la solución primero. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

In [ ]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 35 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [ ]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [ ]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

### Ver el estado de creación de la solución

Según lo prometido, veremos cómo se pueden observar las actualizaciones de estado en la consola:

* En otra pestaña del navegador, ya debería tener la consola de AWS activa desde la apertura de esta instancia de cuaderno. 
* Cambie a esa pestaña y busque en la parte superior el servicio `Personalize`, luego diríjase a la página de ese servicio. 
* Haga clic en `View dataset groups` (Ver grupos de conjuntos de datos).
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente contenga POC en el nombre.
* Haga clic en `Solutions and recipes` (Ver grupos de conjuntos de datos).
* Ahora verá una lista de todas las soluciones que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que esté `Active` (ACTIVA), su solución estará lista para la revisión. También es capaz de implementarse.

O simplemente ejecute la celda a continuación para seguir el estado de creación de la versión de la solución.

In [ ]:
in_progress_solution_versions = [
    userpersonalization_solution_version_arn,
    sims_solution_version_arn,
    rerank_solution_version_arn
]

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

### Ajuste de hiperparámetros

Personalize ofrece la opción de ejecutar el ajuste de hiperparámetros cuando crea una solución. Debido al cálculo adicional necesario para realizar el ajuste de los hiperparámetros, esta función está desactivada por defecto. Por lo tanto, las soluciones que creamos anteriormente, simplemente utilizarán los valores por defecto de los hiperparámetros para cada receta. Para obtener más información sobre el ajuste de los hiperparámetros, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

Si optó por la receta correcta a utilizar y está listo para ejecutar el ajuste de hiperparámetros, el siguiente código muestra cómo lo haría, utilizando SIMS como ejemplo.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-hpo",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    performHPO=True
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

Si ya conoce los valores que desea utilizar para un hiperparámetro específico, también puede establecer este valor al crear la solución. El siguiente código muestra cómo se puede establecer el valor de `popularity_discount_factor` para la receta de SIMS.

```python
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims-set-hp",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn,
    solutionConfig = {
        'algorithmHyperParameters': {
            'popularity_discount_factor': '0.7'
        }
    }
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))
```

## Evaluar las versiones de las soluciones<a class="anchor" id="eval"></a>
[Regresar al principio](#top)

No debería tardar más de una hora en entrenar todas las soluciones de este cuaderno. Mientras se realiza el entrenamiento, se recomienda dedicar tiempo a leer en detalle los distintos algoritmos (recetas) y su comportamiento. Este es también un buen momento para considerar alternativas a la forma en que los datos se introdujeron en el sistema y el tipo de resultados que espera ver.

Cuando las soluciones terminan de crearse, el siguiente paso es obtener las métricas de evaluación. Personalize calcula estas métricas basándose en un subconjunto de los datos de entrenamiento. La siguiente imagen muestra cómo Personalize divide los datos. Dados 10 usuarios, con 10 interacciones cada uno (un círculo representa una interacción), las interacciones se ordenan de la más antigua a la más reciente en función de la marca temporal. Personalize utiliza todos los datos de interacción del 90 % de los usuarios (círculos azules) para entrenar la versión de la solución, y el 10 % restante para la evaluación. Para cada uno de los usuarios del 10 % restante, el 90 % de sus datos de interacción (círculos verdes) se utiliza como entrada para la llamada al modelo entrenado. El 10 % restante de sus datos (círculo naranja) se compara con la salida producida por el modelo y se utiliza para calcular las métricas de evaluación.

![métricas de personalize](static/imgs/personalize_metrics.png)

Recomendamos leer [la documentación](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) para comprender las métricas, pero, además, copiamos partes de la documentación a continuación para mayor comodidad.

Es necesario que comprenda los siguientes términos relativos a la evaluación en Personalize:

* *Recomendación relevante* se refiere a una recomendación que coincide con un valor en los datos de prueba para el usuario en particular.
* *Rango* se refiere a la posición de un elemento recomendado en la lista de recomendaciones. Se supone que la posición 1 (la primera de la lista) es la más relevante para el usuario.
* *Consulta* se refiere al equivalente interno de una llamada a GetRecommendations.

Las métricas producidas por Personalize son las siguientes:
* **coverage**: La proporción de elementos únicos recomendados de todas las consultas sobre el número total de elementos únicos en los datos de entrenamiento (incluye tanto los conjuntos de datos de elementos como de interacciones).
* **mean_reciprocal_rank_at_25**: La [media de los rangos recíprocos](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la primera recomendación relevante entre las 25 primeras recomendaciones sobre todas las consultas. Esta métrica es apropiada si está interesado en la recomendación de mayor rango.
* **normalized_discounted_cumulative_gain_at_K**: La ganancia descontada supone que las recomendaciones más bajas de una lista de recomendaciones son menos relevantes que las más altas. Por lo tanto, cada recomendación se descuenta (se le da un peso menor) por un factor que depende de su posición. A fin de obtener la [ganancia descontada acumulada](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) en K, se suma cada recomendación descontada relevante en las  recomendaciones más importantes de K. La ganancia acumulada descontada normalizada (NDCG) es la DCG dividida por la DCG ideal, de manera que la NDCG está entre 0 y 1. (El DCG ideal es aquel en el que las recomendaciones más importantes de K están ordenadas por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/log (1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica recompensa los elementos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.
* **precision_at_K**: El número de recomendaciones relevantes de las principales recomendaciones de K dividido por K. Esta métrica recompensa la recomendación precisa de los elementos relevantes.

Veamos las métricas de evaluación de cada una de las soluciones elaboradas en este cuaderno. *Tenga en cuenta que sus resultados pueden diferir de los descritos en este cuaderno debido a la calidad del conjunto de datos de Movielens.* 

### Métricas de personalización del usuario

En primer lugar, recupere las métricas de evaluación para la versión de la solución de personalización de usuarios.

In [ ]:
user_personalization_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = userpersonalization_solution_version_arn
)

print(json.dumps(user_personalization_solution_metrics_response, indent=2))

La ganancia acumulada descontada normalizada que se menciona anteriormente nos dice que a 5 elementos tenemos menos de una probabilidad del (38 % para los completos 22 % para los pequeños) en una recomendación que forma parte del historial de interacción de un usuario (en la fase de retención del entrenamiento y la validación). Alrededor del 13 % de los elementos recomendados son únicos, y tenemos una precisión de solo (14 % para los completos, 7,5 % para los pequeños) en los 5 primeros elementos recomendados. 

Está claro que no es un gran modelo, pero hay que tener en cuenta que hemos tenido que utilizar datos de valoración para nuestras interacciones porque MovieLens es un conjunto de datos explícito basado en valoraciones. Las marcas temporales también son del momento en que se calificó la película, no del momento en que se vio, por lo que el orden no es el mismo en que un espectador vería las películas.

### Métricas SIMS

Ahora, recupere las métricas de evaluación para la versión de la solución SIMS.

In [ ]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

En este ejemplo, vemos una precisión ligeramente elevada a 5 elementos, un poco más del (4,5 % para los completos, 6,4 % para los pequeños) esta vez. Efectivamente, esto está quizás dentro del margen de error, pero dado que no se ha hecho ningún esfuerzo por enmascarar la popularidad, puede ser que simplemente se devuelvan resultados muy populares con los que un gran volumen de usuarios ha interactuado de alguna manera. 

### Métricas de clasificación personalizadas

Ahora, recupere las métricas de evaluación para la versión de la solución de clasificación personalizada.

In [ ]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

Simplemente un comentario rápido sobre esto, aquí vemos de nuevo una precisión de cerca del (2,7 % para el completo, 2,2 % para el pequeño), ya que esto se basa en la personalización del usuario, que es de esperar. Sin embargo, los elementos de la muestra no son los mismos para la validación, de ahí la baja puntuación.

## Uso de métricas de evaluación <a class="anchor" id="use"></a>
[Regresar al principio](#top)

Es importante utilizar cuidadosamente los parámetros de evaluación. Es necesario tener en cuenta una serie de factores.

* Si ya existe un sistema de recomendación, este habrá influido en el historial de interacción del usuario, que se utilizará para entrenar las nuevas soluciones. Esto significa que las métricas de evaluación están condicionadas a favorecer la solución existente. Si usted procura que las métricas de evaluación sean iguales o superiores a las de la solución existente, es posible que sólo esté empujando la personalización del usuario para que se comporte como la solución existente y que no termine con algo mejor.
* La receta HRNN Coldstart es difícil de evaluar con las métricas producidas por Amazon Personalize. El objetivo de la receta es recomendar elementos nuevos para su negocio. Por lo tanto, estos elementos no aparecerán en los datos de transacciones de los usuarios existentes que se utilizan para calcular las métricas de evaluación. Como resultado, cuando se comparan únicamente las métricas de la evaluación, no se podrá observar que el Coldstart de HRNN tenga un mejor rendimiento que las otras recetas. Nota: La receta de personalización del usuario también incluye una funcionalidad mejorada de arranque en frío.

Teniendo en cuenta estos factores, las métricas de evaluación producidas por Personalize son generalmente útiles para dos casos:
1. La comparación del rendimiento de las versiones de la solución entrenadas con la misma receta, pero con diferentes valores para los hiperparámetros y las características (datos de impresión, etc.).
1. La comparación del rendimiento de las versiones de la solución entrenadas en diferentes recetas (excepto HRNN Coldstart).

La mejor forma de evaluar adecuadamente un sistema de recomendación es siempre a través de pruebas A/B mientras se miden los resultados reales del negocio. Dado que las recomendaciones generadas por un sistema suelen influir en el comportamiento del usuario en el que se basan, es mejor realizar pequeños experimentos y aplicar pruebas A/B durante periodos de tiempo más largos. Con el tiempo, el predominio del modelo existente disminuirá.

## Almacenamiento de variables útiles <a class="anchor" id="vars"></a>
[Regresar al principio](#top)

Antes de salir de este cuaderno, ejecute las siguientes celdas para guardar los ARN de la versión y utilizarlos en el siguiente cuaderno.

In [ ]:
%store userpersonalization_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn
%store user_personalization_solution_arn
%store sims_solution_arn
%store rerank_solution_arn